In [ ]:
from typing import TypedDict


class GraphState(TypedDict):
    number_1: int
    number_2: int
    operation: str
    output: int


# Edge names
ADDER_EDGE: str = "ADDER_EDGE"
SUBTRACTER_EDGE: str = "SUBTRACTER_EDGE"

In [ ]:
def adder_node(state: GraphState) -> GraphState:
    """Simple node that add two number and update the output into state"""

    state["output"] = state["number_1"] + state["number_2"]
    return state


def subtracter_node(state: GraphState) -> GraphState:
    """Simple node that subtract 2nd number from 1st number and update the output into state"""

    state["output"] = state["number_1"] - state["number_2"]
    return state


def decide_next_node(state: GraphState):
    """This simple node will be selected next node of the graph"""

    match state["operation"]:
        case "+":
            return ADDER_EDGE
        case "-":
            return SUBTRACTER_EDGE
        case _:
            return None

In [ ]:
from langgraph.graph import StateGraph, START, END

ROUTER_NODE: str = "ROUTER"
ADDER_NODE: str = "ADDER"
SUBTRACTER_NODE: str = "SUBTRACTER"

state_graph = StateGraph(GraphState)
state_graph.add_node(ROUTER_NODE, lambda state: state)  # passthrough function
state_graph.add_node(ADDER_NODE, adder_node)
state_graph.add_node(SUBTRACTER_NODE, subtracter_node)

state_graph.add_edge(START, ROUTER_NODE)
state_graph.add_conditional_edges(
    ROUTER_NODE,
    decide_next_node,
    {ADDER_EDGE: ADDER_NODE, SUBTRACTER_EDGE: SUBTRACTER_NODE},
)
state_graph.add_edge(ADDER_NODE, END)
state_graph.add_edge(SUBTRACTER_NODE, END)

graph = state_graph.compile()
graph

In [ ]:
initial_state = GraphState(number_1=10, number_2=20, operation="+", output=0)
print(graph.invoke(initial_state))

In [ ]:
initial_state = GraphState(number_1=30, number_2=20, operation="-", output=0)
print(graph.invoke(initial_state))
